In [28]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
tracks = pd.read_csv('F:\ML\Music genre/fma-rock-vs-hiphop.csv')
music_features = pd.read_json('F:\ML\Music genre/echonest-metrics.json',precise_float=True)
# tracks.head(6)
# music_features.head(5)
audio_data=music_features.merge(tracks[['genre_top', 'track_id']], on='track_id')
# audio_data.head(5)
# audio_data.info()
# correlations = audio_data.corr()
# correlations.style.background_gradient()


features = audio_data.drop(['genre_top','track_id'],axis=1)
labels = audio_data['genre_top']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
# pca = PCA()
# pca.fit(scaled_features)
# exp_variance = pca.explained_variance_ratio_
# num_components = pca.n_components_
# fig, ax = plt.subplots()
# ax.bar(range(num_components), exp_variance)
# ax.set_xlabel('Principal Component #')


# cum_exp_variance = np.cumsum(exp_variance) #calculate the cumulative explained variance

# fig, ax = plt.subplots()
# ax.plot(cum_exp_variance)
# ax.axhline(y=0.85, linestyle='--') #for considering features needed to explain 85% of the variance


n_components = 6
pca = PCA(n_components, random_state=10)
pca.fit(scaled_features)
pca_projection = pca.transform(scaled_features)
pca_projection.shape

# print(pca_projection)
#Decision Tree Classifier
X_train, X_test, y_train, y_test = train_test_split(pca_projection,labels,random_state=10)
model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train,y_train)
predictions_dt = model_dt.predict(X_test)
# print("Decision Tree Classifier:", model_dt.score(X_test,y_test))

#Logistic regression
model_lg = LogisticRegression(random_state=10)
model_lg.fit(X_train,y_train)
predictions_lg = model_lg.predict(X_test)
# print("Logistic Regression:", model_lg.score(X_test,y_test))

report_dt = classification_report(y_test,predictions_dt)
report_lg = classification_report(y_test,predictions_lg)

# print("Decision Tree Classifier: \n", report_dt)
# print("Logistic Regression: \n", report_lg)

# sns.countplot(audio_data['genre_top'], label = "Count") 

hip_hop = audio_data.loc[audio_data['genre_top'] == 'Hip-Hop']
rock = audio_data.loc[audio_data['genre_top'] == 'Rock']
# hip_hop.shape
# rock.shape

rock = rock.sample(hip_hop.shape[0], random_state=10)
# # rock.shape

balanced_data = pd.concat([rock, hip_hop])

features = balanced_data.drop(['genre_top', 'track_id'], axis=1) 
labels = balanced_data['genre_top']
pca_projection = pca.fit_transform(scaler.fit_transform(features))
# pca_projection.shape

X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train,y_train)
predictions_dt = model_dt.predict(X_test)

model_lr = LogisticRegression(random_state=10)
model_lr.fit(X_train,y_train)
predictions_lr = model_lr.predict(X_test)

# print("Decision Tree: \n", classification_report(y_test, predictions_dt))
# print("Logistic Regression: \n", classification_report(y_test, predictions_lr))

cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(model_dt, X_test,y_test, scoring='accuracy', cv=cv, n_jobs=-1)
print("Decision Tree Classifier Accuracy:", np.mean((scores)))
scores = cross_val_score(model_lg, X_test,y_test, scoring='accuracy', cv=cv, n_jobs=-1)
print("Logistic Regression Accuracy:", np.mean((scores)))

Decision Tree Classifier Accuracy: 0.7979227053140097
Logistic Regression Accuracy: 0.8157971014492755


In [29]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score

# Load data
tracks = pd.read_csv('F:\ML\Music genre/fma-rock-vs-hiphop.csv')
music_features = pd.read_json('F:\ML\Music genre/echonest-metrics.json', precise_float=True)

# Merge data
audio_data = music_features.merge(tracks[['genre_top', 'track_id']], on='track_id')

# Prepare features and labels
features = audio_data.drop(['genre_top', 'track_id'], axis=1)
labels = audio_data['genre_top']

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# PCA transformation
n_components = 6
pca = PCA(n_components, random_state=10)
pca_projection = pca.fit_transform(scaled_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

# Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train, y_train)
predictions_dt = model_dt.predict(X_test)

# Logistic Regression
model_lg = LogisticRegression(random_state=10)
model_lg.fit(X_train, y_train)
predictions_lg = model_lg.predict(X_test)

# Classification reports
report_dt = classification_report(y_test, predictions_dt)
report_lg = classification_report(y_test, predictions_lg)
print("Decision Tree Classifier:\n", report_dt)
print("Logistic Regression:\n", report_lg)

# Balance the dataset
hip_hop = audio_data.loc[audio_data['genre_top'] == 'Hip-Hop']
rock = audio_data.loc[audio_data['genre_top'] == 'Rock']
rock = rock.sample(hip_hop.shape[0], random_state=10)
balanced_data = pd.concat([rock, hip_hop])

# Prepare balanced features and labels
features = balanced_data.drop(['genre_top', 'track_id'], axis=1)
labels = balanced_data['genre_top']
pca_projection = pca.fit_transform(scaler.fit_transform(features))

# Train-test split on balanced data
X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

# Decision Tree Classifier on balanced data
model_dt.fit(X_train, y_train)
predictions_dt = model_dt.predict(X_test)

# Logistic Regression on balanced data
model_lr = LogisticRegression(random_state=10)
model_lr.fit(X_train, y_train)
predictions_lr = model_lr.predict(X_test)

# Classification reports on balanced data
print("Decision Tree on balanced data:\n", classification_report(y_test, predictions_dt))
print("Logistic Regression on balanced data:\n", classification_report(y_test, predictions_lr))

# Cross-validation
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores_dt = cross_val_score(model_dt, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
print("Decision Tree Classifier Accuracy (cross-validated):", np.mean(scores_dt))
scores_lg = cross_val_score(model_lg, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
print("Logistic Regression Accuracy (cross-validated):", np.mean(scores_lg))


Decision Tree Classifier:
               precision    recall  f1-score   support

     Hip-Hop       0.60      0.60      0.60       235
        Rock       0.90      0.90      0.90       966

    accuracy                           0.84      1201
   macro avg       0.75      0.75      0.75      1201
weighted avg       0.84      0.84      0.84      1201

Logistic Regression:
               precision    recall  f1-score   support

     Hip-Hop       0.77      0.54      0.64       235
        Rock       0.90      0.96      0.93       966

    accuracy                           0.88      1201
   macro avg       0.83      0.75      0.78      1201
weighted avg       0.87      0.88      0.87      1201

Decision Tree on balanced data:
               precision    recall  f1-score   support

     Hip-Hop       0.74      0.73      0.74       230
        Rock       0.73      0.74      0.73       225

    accuracy                           0.74       455
   macro avg       0.74      0.74      0.74   

In [30]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score

# Load data
tracks = pd.read_csv('F:\ML\Music genre/fma-rock-vs-hiphop.csv')
music_features = pd.read_json('F:\ML\Music genre/echonest-metrics.json', precise_float=True)

# Merge data
audio_data = music_features.merge(tracks[['genre_top', 'track_id']], on='track_id')

# Prepare features and labels
features = audio_data.drop(['genre_top', 'track_id'], axis=1)
labels = audio_data['genre_top']

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# PCA transformation
n_components = 6
pca = PCA(n_components, random_state=10)
pca_projection = pca.fit_transform(scaled_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

# Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train, y_train)

# Logistic Regression
model_lg = LogisticRegression(random_state=10)
model_lg.fit(X_train, y_train)

# Function to predict music genre based on user input
def predict_genre(input_features):
    input_df = pd.DataFrame([input_features], columns=features.columns)
    scaled_input = scaler.transform(input_df)
    pca_input = pca.transform(scaled_input)

    dt_prediction = model_dt.predict(pca_input)[0]
    lg_prediction = model_lg.predict(pca_input)[0]
    
    return dt_prediction, lg_prediction

# Example usage:
# Replace the values in the dictionary below with the actual features for a new song
new_song_features = {
    "acousticness": 0.5,
    "danceability": 0.6,
    "energy": 0.7,
    "instrumentalness": 0.0,
    "liveness": 0.1,
    "speechiness": 0.05,
    "valence": 0.4,
    "tempo": 120.0,
    # Add all other necessary features here
}

# Get predictions
dt_genre, lg_genre = predict_genre(new_song_features)
print("Decision Tree Prediction:", dt_genre)
print("Logistic Regression Prediction:", lg_genre)


Decision Tree Prediction: Rock
Logistic Regression Prediction: Rock


In [31]:
pip install librosa

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Wav file

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
import librosa

# Load data
tracks = pd.read_csv('F:\\ML\\Music genre\\fma-rock-vs-hiphop.csv')
music_features = pd.read_json('F:\\ML\\Music genre\\echonest-metrics.json', precise_float=True)

# Merge data
audio_data = music_features.merge(tracks[['genre_top', 'track_id']], on='track_id')

# Prepare features and labels
features = audio_data.drop(['genre_top', 'track_id'], axis=1)
labels = audio_data['genre_top']

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# PCA transformation
n_components = 6
pca = PCA(n_components, random_state=10)
pca_projection = pca.fit_transform(scaled_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

# Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train, y_train)

# Logistic Regression
model_lg = LogisticRegression(random_state=10)
model_lg.fit(X_train, y_train)

# Function to extract features from an audio file using librosa
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    features = {
        "acousticness": np.mean(librosa.feature.zero_crossing_rate(y)),
        "danceability": np.mean(librosa.feature.chroma_stft(y, sr=sr)),
        "energy": np.mean(librosa.feature.rms(y)),
        "instrumentalness": np.mean(librosa.feature.spectral_centroid(y, sr=sr)),
        "liveness": np.mean(librosa.feature.spectral_bandwidth(y, sr=sr)),
        "speechiness": np.mean(librosa.feature.spectral_contrast(y, sr=sr)),
        "valence": np.mean(librosa.feature.tonnetz(y, sr=sr)),
        "tempo": librosa.beat.tempo(y, sr=sr)[0]
    }
    return features

# Function to predict music genre based on audio file
def predict_genre_from_audio(file_path):
    input_features = extract_features(file_path)
    input_df = pd.DataFrame([input_features], columns=features.columns)
    scaled_input = scaler.transform(input_df)
    pca_input = pca.transform(scaled_input)

    dt_prediction = model_dt.predict(pca_input)[0]
    lg_prediction = model_lg.predict(pca_input)[0]
    
    return dt_prediction, lg_prediction

# Example usage:
audio_file_path = r'C:\Users\dell\Downloads\puspa.mp3'
dt_genre, lg_genre = predict_genre_from_audio(audio_file_path)
print("Decision Tree Prediction:", dt_genre)
print("Logistic Regression Prediction:", lg_genre)


TypeError: chroma_stft() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

In [6]:
# mp3 file

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
import librosa

# Load data
tracks = pd.read_csv('F:\\ML\\Music genre\\fma-rock-vs-hiphop.csv')
music_features = pd.read_json('F:\\ML\\Music genre\\echonest-metrics.json', precise_float=True)

# Merge data
audio_data = music_features.merge(tracks[['genre_top', 'track_id']], on='track_id')

# Prepare features and labels
features = audio_data.drop(['genre_top', 'track_id'], axis=1)
labels = audio_data['genre_top']

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# PCA transformation
n_components = 6
pca = PCA(n_components, random_state=10)
pca_projection = pca.fit_transform(scaled_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(pca_projection, labels, random_state=10)

# Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=10)
model_dt.fit(X_train, y_train)

# Logistic Regression
model_lg = LogisticRegression(random_state=10)
model_lg.fit(X_train, y_train)

# Function to extract features from an audio file using librosa
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    features = {
        "acousticness": np.mean(librosa.feature.zero_crossing_rate(y)),
        "danceability": np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
        "energy": np.mean(librosa.feature.rms(y=y)),
        "instrumentalness": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        "liveness": np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
        "speechiness": np.mean(librosa.feature.spectral_contrast(y=y, sr=sr)),
        "valence": np.mean(librosa.feature.tonnetz(y=y, sr=sr)),
        "tempo": librosa.beat.tempo(y=y, sr=sr)[0]
    }
    return features

# Function to predict music genre based on audio file
def predict_genre_from_audio(file_path):
    input_features = extract_features(file_path)
    input_df = pd.DataFrame([input_features])
    
    # Ensure feature columns match
    input_df = input_df.reindex(columns=features.columns, fill_value=0)
    
    scaled_input = scaler.transform(input_df)
    pca_input = pca.transform(scaled_input)

    dt_prediction = model_dt.predict(pca_input)[0]
    lg_prediction = model_lg.predict(pca_input)[0]
    
    return dt_prediction, lg_prediction

# Example usage:
audio_file_path = r'C:\Users\dell\Downloads\hip hop.mp3'
dt_genre, lg_genre = predict_genre_from_audio(audio_file_path)
print("Decision Tree Prediction:", dt_genre)
print("Logistic Regression Prediction:", lg_genre)


Decision Tree Prediction: Rock
Logistic Regression Prediction: Rock


C:\Users\dell\AppData\Local\Temp\ipykernel_7992\1228776820.py:58: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  "tempo": librosa.beat.tempo(y=y, sr=sr)[0]
